In [1]:
# from __future__ import print_function

import sys

import numpy as np
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import pyspark.sql.functions as F

import pytz

# print(datetime.now(pytz.timezone("Asia/Shanghai")).strftime( '%Y-%m-%d'))
#北京时间0点半执行，相当于utc时间昨天的16:30 ，所以时间不-1天
# yesterday = datetime.now().strftime('%Y-%m-%d')
# yesterday = datetime.now().strftime('%Y-%m-%d')
# partitionlist=yesterday.split("-",3)
# beforeyesterday = (datetime.now() + timedelta(-1)).strftime('%Y-%m-%d')
# #防止当天出错之后,八点之后运行会丢失分区
# partitionlist.append(beforeyesterday.split("-",3)[2])
#是否特殊special，特殊的需要自己写sql
# print()builtins
if __name__ == "__main__":
    spark = SparkSession \
        .builder \
        .appName("Python Demo") \
        .config("hive.metastore.client.factory.class",
                "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory") \
        .config("spark.driver.maxResultSize", "4g") \
        .enableHiveSupport() \
        .getOrCreate()
    spark.conf.set("hive.exec.dynamic.partition.mode", "nonstrict");
    spark.conf.set("spark.sql.sources.partitionOverwriteMode", "dynamic")
    databases = spark.sql("show databases")
    databases = databases.collect()
#     for  row in tablelist:
#         #添加分区 
# #       spark.sql(partitionsql)
#         partitionsql_1.replace("#table#", row) 
#         spark.sql(partitionsql_1)
#         partitionsql_2.replace("#table#", row) 
#         spark.sql(partitionsql_2)
    df1=spark.sql("""
        with
            parent_table as (
              -- 获取开始展期的第一笔loan
              select 
              id loan_id,
              customer_id
              from `luzon_etl_s3`.t_loan_app
              where paid_off_mode = 'ROLLOVER' and (parent_id is null or parent_id = 0) 
              and date(create_time )  >= date '2019-01-01'

              ),
            last_roll_table as (
              -- 获取当前展期周期的最后一笔rollover，最后一笔的状态只能是('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT')
              select
              last_loan_id,
              status,
              customer_id
              from (
              select
              l.id last_loan_id,
              l.status,
              l.customer_id,
              rank() over (partition by l.customer_id order by p.loan_id) rnk
              from `luzon_etl_s3`.t_loan_app l
              right join parent_table p
              on l.customer_id = p.customer_id and p.loan_id < l.id and (l.paid_off_mode is null and l.status in ('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT'))
              where date(l.create_time)  >= date '2019-01-01'
              ) rnk
              where rnk.rnk = 1
                ),
            rollover_table as (    
              -- 获取所有rollover loan，且状态为('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT')
              -- 同一个customer下的，所有阶段的rollover loan，一定满足：parent_loan_id < rollover_loan_id <= last_rollover_loan_id
            select id,customer_id,parent_loan_id as primary_id ,last_loan_id, rank() over(partition by parent_loan_id order by id) as rollover_rnk from (
              select
              l.id,l.customer_id,
              l.parent_id,
              l.status,
              l.loan_type,
              l.paid_off_mode,
              rank() over (partition by l.id order by p.loan_id desc, lr.last_loan_id) rnk,
              p.loan_id parent_loan_id,
              lr.last_loan_id
              from `luzon_etl_s3`.t_loan_app l
              right join parent_table p
              on p.loan_id < l.id and p.customer_id = l.customer_id 
              left join last_roll_table lr
              on l.id <= lr.last_loan_id and lr.customer_id = l.customer_id
              where 
                l.status in ('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT') and date(l.create_time  )  >= date '2019-01-01'
                and l.loan_type = 'ROLLOVER'
              order by l.id
                )
                where rnk = 1
                order by id
              ),
            last_loan as 
            (
              select primary_id ,last_loan_id,count(*) as k
              from rollover_table
              group by primary_id ,last_loan_id
            ),
            issue as 
            (
              select app.id,min(status.create_time) as issue_time
              from `luzon_etl_s3`.t_loan_app app
              left join `luzon_etl_s3`.t_loan_app_status_log status
              on app.id = status.loan_app_id
              where app.status in ('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT') and status.new_status = 'CURRENT'
              and date(app.create_time   )  >= date '2019-01-01'
              group by app.id
            ),
            paid_off as 
            (
              select app.id,min(status.create_time) as paid_off_time
              from `luzon_etl_s3`.t_loan_app app
              left join `luzon_etl_s3`.t_loan_app_status_log status
              on app.id = status.loan_app_id
              where app.status in ('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT') and status.new_status = 'PAID_OFF'
              and date(app.create_time )  >= date '2019-01-01'
              group by app.id
            ),
            issue_rank as 
            (
              select app.id,rank() over (partition by customer_id order by id) as rnk
              from `luzon_etl_s3`.t_loan_app app
              where app.status in ('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT')  and app.loan_type != 'ROLLOVER'
              and date(app.create_time  + interval '7' hour )  >= date '2019-08-01'
            )
            select app.id,app.amount as issue_amount,app.status,app.create_time,lpay.due_date,app.customer_id,
                   CASE WHEN lpay.principal_paid IS NULL THEN 0 ELSE lpay.principal_paid END  as principal_paid,
                   CASE WHEN lpay.interest_paid IS NULL THEN 0 ELSE lpay.interest_paid END as interest_paid,
                   CASE WHEN lpay.default_paid IS NULL THEN 0 ELSE lpay.default_paid END as default_paid,
                   CASE WHEN lpay.service_fee_paid IS NULL THEN 0 ELSE lpay.service_fee_paid END as service_fee_paid,
                   CASE WHEN lpay.rollover_fee_paid IS NULL THEN 0 ELSE lpay.rollover_fee_paid END as rollover_fee_paid,
                   CASE WHEN lpay.service_fee_accr IS NULL THEN 0 ELSE lpay.service_fee_accr END as default_fee_accr,
                   CASE WHEN lpay.erase_amount  IS NULL THEN 0 ELSE lpay.erase_amount  END as erase_amount,
                   app.loan_type,app.paid_off_mode,app.parent_id,
                   case when app.loan_type != 'ROLLOVER' then app.id else roll.primary_id end  as primary_id,
                   case when app.paid_off_mode is null then app.id when ll.last_loan_id is not null then ll.last_loan_id else roll.last_loan_id end as last_loan_id,
                   case when app.loan_type != 'ROLLOVER' then 0 else roll.rollover_rnk end as rollover_rnk,ie.issue_time,po.paid_off_time,
                   ik.rnk as issue_rnk
            from `luzon_etl_s3`.t_loan_app app
            left join `luzon_etl_s3`.t_lpay lpay
            on app.id = lpay.loan_app_id
            left join rollover_table roll
            on app.id = roll.id
            left join last_loan ll
            on app.id = ll.primary_id
            left join issue ie
            on app.id = ie.id
            left join paid_off po
            on app.id = po.id
            left join issue_rank as ik
            on app.id = ik.id
            where app.status in ('PAID_OFF', 'OVERDUE', 'GRACE_PERIOD', 'CURRENT') and date(app.create_time )  >= date '2019-01-01'

    """)
    df1.write.mode("overwrite").orc("s3://rupiahplus-data-warehouse/etl/luzon/analysis/t_loan_eft")



VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1591930254723_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…